In [ ]:
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import json
import os
import getpass
from typing import Optional, Literal

# AI Integration Imports
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import END, MessagesState, StateGraph, START
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from huggingface_hub import InferenceClient  # For HuggingFace integration
import requests  # For direct DeepSeek API fallback

In [ ]:
# Configuration Setup
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}: ")

_set_if_undefined("BINANCE_API_KEY")
_set_if_undefined("BINANCE_API_SECRET")
_set_if_undefined("HF_API_KEY")  # HuggingFace API key
_set_if_undefined("DEEPSEEK_API_KEY")  # Fallback DeepSeek API key

# Initialize Binance exchange
exchange = ccxt.binance({
    'apiKey': os.environ["BINANCE_API_KEY"],
    'secret': os.environ["BINANCE_API_SECRET"],
    'enableRateLimit': True,
})

In [ ]:
# AI Client Setup
class AIClient:
    def __init__(self):
        self.hf_client = InferenceClient(
            provider="nebius",
            api_key=os.environ["HF_API_KEY"],
        )
        self.langchain_llm = ChatOpenAI(
            model="deepseek-chat",
            api_key=os.environ["DEEPSEEK_API_KEY"],
            base_url="https://api.deepseek.com/v1"
        )
    
    async def get_ai_response(self, messages: list, model: str = "deepseek-ai/DeepSeek-V3-0324"):
        """Dual-mode AI inference with fallback"""
        try:
            # First try HuggingFace inference
            completion = self.hf_client.chat.completions.create(
                model=model,
                messages=messages,
                max_tokens=512,
            )
            return completion.choices[0].message.content
        except Exception as hf_error:
            print(f"HuggingFace error: {hf_error}, falling back to DeepSeek API")
            
            # Fallback to LangChain DeepSeek
            prompt = ChatPromptTemplate.from_messages(messages)
            chain = prompt | self.langchain_llm
            return chain.invoke({"messages": messages}).content

ai_client = AIClient()

In [ ]:
# Crypto Data Functions
async def get_crypto_data(symbol: str, timeframe: str = '1d', limit: int = 100) -> pd.DataFrame:
    """Fetch OHLCV data from Binance"""
    try:
        ohlcv = await exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        return df
    except Exception as e:
        raise Exception(f"Binance API Error: {str(e)}")

def calculate_crypto_signals(data: pd.DataFrame) -> dict:
    """Calculate crypto-specific technical indicators"""
    # Simple Moving Averages
    data['sma_20'] = data['close'].rolling(20).mean()
    data['sma_50'] = data['close'].rolling(50).mean()
    
    # RSI
    delta = data['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    data['rsi'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    data['upper_band'] = data['sma_20'] + (2 * data['close'].rolling(20).std())
    data['lower_band'] = data['sma_20'] - (2 * data['close'].rolling(20).std())
    
    return {
        'current_price': data['close'].iloc[-1],
        'sma_20': data['sma_20'].iloc[-1],
        'sma_50': data['sma_50'].iloc[-1],
        'rsi': data['rsi'].iloc[-1],
        'upper_band': data['upper_band'].iloc[-1],
        'lower_band': data['lower_band'].iloc[-1],
        'volume': data['volume'].iloc[-1]
    }

In [ ]:
# Agent Nodes
async def market_data_agent(state: MessagesState):
    """Gathers and processes crypto market data"""
    messages = state["messages"]
    params = messages[-1].additional_kwargs
    
    # Get crypto data
    data = await get_crypto_data(
        params["symbol"],
        params.get("timeframe", "1d"),
        params.get("limit", 100)
    )
    
    signals = calculate_crypto_signals(data)
    
    message = HumanMessage(
        content=f"""
        {params["symbol"]} Crypto Signals:
        Current Price: {signals['current_price']:.5f}
        SMA 20: {signals['sma_20']:.5f}
        SMA 50: {signals['sma_50']:.5f}
        RSI: {signals['rsi']:.2f}
        Upper Band: {signals['upper_band']:.5f}
        Lower Band: {signals['lower_band']:.5f}
        Volume: {signals['volume']:.2f}
        """,
        name="market_data_agent",
    )
    
    return {"messages": messages + [message]}

async def quant_agent(state: MessagesState):
    """Analyzes crypto technical indicators"""
    last_message = state["messages"][-1]
    
    messages = [
        SystemMessage(content="""You are a crypto trading expert analyzing market data.
        Provide analysis with:
        - signal: bullish | bearish | neutral
        - confidence: <float 0-1>
        - reasoning: <brief explanation>
        Consider crypto volatility and 24/7 markets."""),
        last_message
    ]
    
    result = await ai_client.get_ai_response(messages)
    return {"messages": state["messages"] + [
        HumanMessage(content=f"Crypto Analysis: {result}", name="quant_agent")
    ]}

async def risk_management_agent(state: MessagesState):
    """Crypto-specific risk management"""
    portfolio = state["messages"][0].additional_kwargs["portfolio"]
    last_message = state["messages"][-1]
    
    messages = [
        SystemMessage(content="""Crypto Risk Manager:
        - Max position: 1-5% of capital
        - Consider volatility (higher for crypto)
        - Account for potential flash crashes
        Output: max_position_size, risk_score(1-10)"""),
        MessagesPlaceholder(variable_name="messages"),
        HumanMessage(content=f"""Analysis: {last_message.content}
        Portfolio: {portfolio['cash']:.2f} USDT | Position: {portfolio['position']:.8f} BTC""")
    ]
    
    result = await ai_client.get_ai_response(messages)
    return {"messages": state["messages"] + [
        HumanMessage(content=f"Risk Assessment: {result}", name="risk_management")
    ]}

async def portfolio_management_agent(state: MessagesState):
    """Executes crypto trades with proper sizing"""
    portfolio = state["messages"][0].additional_kwargs["portfolio"]
    last_message = state["messages"][-1]
    
    messages = [
        SystemMessage(content="""Crypto Portfolio Manager:
        - Minimum order sizes (check exchange)
        - Action: buy | sell | hold
        - Quantity: in coin units
        - Maintain 1-2% risk per trade (higher crypto volatility)
        - Consider maker/taker fees"""),
        last_message,
        HumanMessage(content=f"""Risk Data: {last_message.content}
        Portfolio: {portfolio['cash']:.2f} USDT | Position: {portfolio['position']:.8f} BTC""")
    ]
    
    result = await ai_client.get_ai_response(messages)
    return {"messages": [HumanMessage(content=result, name="portfolio_management")]}


In [ ]:
# Workflow Construction
workflow = StateGraph(MessagesState)
workflow.add_node("market_data_agent", market_data_agent)
workflow.add_node("quant_agent", quant_agent)
workflow.add_node("risk_management_agent", risk_management_agent)
workflow.add_node("portfolio_management_agent", portfolio_management_agent)

workflow.add_edge(START, "market_data_agent")
workflow.add_edge("market_data_agent", "quant_agent")
workflow.add_edge("quant_agent", "risk_management_agent")
workflow.add_edge("risk_management_agent", "portfolio_management_agent")
workflow.add_edge("portfolio_management_agent", END)

app = workflow.compile()

In [ ]:
# Crypto Backtester
class CryptoBacktester:
    def __init__(self, agent, symbol, start_date, end_date, initial_capital):
        self.agent = agent
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.initial_capital = initial_capital
        self.portfolio = {"cash": initial_capital, "position": 0.0}
        self.portfolio_values = []
        self.last_valid_price = 10000.0  # Reasonable BTC default

    def parse_action(self, output):
        try:
            import re
            action = "hold"
            quantity = 0.0

            action_match = re.search(r'action["\']?\s*:\s*["\']?(\w+)', output, re.IGNORECASE)
            quantity_match = re.search(r'quantity["\']?\s*:\s*([\d.]+)', output, re.IGNORECASE)

            if action_match:
                action = action_match.group(1).lower()
            if quantity_match:
                quantity = float(quantity_match.group(1))

            return action, max(0.0, quantity)
        except:
            return "hold", 0.0

    async def execute_trade(self, action, quantity, current_price):
        """Safe crypto trade execution with validation"""
        if current_price <= 0:
            return 0.0

        # Round to 8 decimals for crypto precision
        quantity = round(quantity, 8)

        if action == "buy":
            max_affordable = self.portfolio["cash"] / current_price
            quantity = min(quantity, max_affordable)
            if quantity > 0:
                self.portfolio["position"] += quantity
                self.portfolio["cash"] -= quantity * current_price
                return quantity
        elif action == "sell":
            quantity = min(quantity, self.portfolio["position"])
            if quantity > 0:
                self.portfolio["position"] -= quantity
                self.portfolio["cash"] += quantity * current_price
                return quantity
        return 0.0

    async def run_backtest(self):
        dates = pd.date_range(self.start_date, self.end_date, freq="D")

        print("\nStarting Crypto Backtest...")
        print(f"{'Date':<12} {'Action':<6} {'Quantity':>12} {'Price':>12} {'Cash':>12} {'Position':>12} {'Value':>12}")
        print("-" * 78)

        for current_date in dates:
            current_date_str = current_date.strftime("%Y-%m-%d")
            lookback_start = (current_date - timedelta(days=30)).strftime("%Y-%m-%d")

            action = "hold"
            executed_qty = 0.0
            current_price = self.last_valid_price
            total_value = self.portfolio["cash"] + self.portfolio["position"] * current_price

            try:
                # Get agent decision
                agent_output = await self.agent({
                    "messages": [HumanMessage(
                        content="Make a trading decision",
                        additional_kwargs={
                            "symbol": self.symbol,
                            "start_date": lookback_start,
                            "end_date": current_date_str,
                            "portfolio": self.portfolio.copy()
                        }
                    )]
                })
                final_output = agent_output["messages"][-1].content

                # Get market data
                df = await get_crypto_data(self.symbol, '1d', 30)
                if len(df) < 1:
                    raise ValueError("No data returned for date range")

                current_price = df.iloc[-1]['close']
                self.last_valid_price = current_price

                # Parse and execute trade
                action, quantity = self.parse_action(final_output)
                executed_qty = await self.execute_trade(action, quantity, current_price)

            except Exception as e:
                print(f"Error processing {current_date_str}: {str(e)}")
                current_price = self.last_valid_price

            total_value = self.portfolio["cash"] + self.portfolio["position"] * current_price

            self.portfolio_values.append({
                "Date": current_date.date(),
                "Portfolio Value": total_value
            })

            print(f"{current_date_str} "
                  f"{action:<6} {executed_qty:>12.8f} {current_price:>12.2f} "
                  f"{self.portfolio['cash']:>12.2f} {self.portfolio['position']:>12.8f} {total_value:>12.2f}")

    def analyze_performance(self):
        if not self.portfolio_values:
            print("No backtest results to analyze")
            return None

        performance_df = pd.DataFrame(self.portfolio_values)
        performance_df['Date'] = pd.to_datetime(performance_df['Date'])
        performance_df.set_index('Date', inplace=True)

        # Handle missing dates
        all_dates = pd.date_range(start=self.start_date, end=self.end_date, freq='D')
        performance_df = performance_df.reindex(all_dates, method='ffill')

        # Calculate metrics
        initial = self.initial_capital
        final = performance_df['Portfolio Value'].iloc[-1]
        total_return = (final - initial) / initial

        print(f"\nBacktest Results ({self.symbol})")
        print(f"Initial Capital: ${initial:,.2f}")
        print(f"Final Value: ${final:,.2f}")
        print(f"Total Return: {total_return*100:.2f}%")

        # Plot performance
        performance_df["Portfolio Value"].plot(title="Crypto Portfolio Performance", figsize=(12,6))
        plt.ylabel("Value (USDT)")
        plt.show()

        return performance_df

# Run the backtest
async def main():
    crypto_backtester = CryptoBacktester(
        agent=app,
        symbol="BTC/USDT",
        start_date="2025-01-01",
        end_date="2025-01-31",
        initial_capital=10000
    )
    
    await crypto_backtester.run_backtest()
    crypto_backtester.analyze_performance()

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())